# Развертывание модели машинного обучения

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

## Предобработка данных

In [2]:
file_path = '../../[3] Logistic_Regression/Churn_prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

df = pd.read_csv(file_path)

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.churn = (df.churn == 'yes').astype(int)

In [3]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

df_train_full = df_train_full.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

y_train = df_train.churn.values
y_val = df_val.churn.values

del df_train['churn']
del df_val['churn']

In [4]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [5]:
def train(df, y, C=1.0):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)

    X = dv.transform(cat)

    model = LogisticRegression(solver='liblinear', C=C)
    model.fit(X, y)

    return dv, model


def predict(df, dv, model):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(cat)

    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [6]:
y_train = df_train_full.churn.values
y_test = df_test.churn.values

dv, model = train(df_train_full, y_train, C=0.5)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
print('auc = %.3f' % auc)

auc = 0.858


In [7]:
# Воспользуемся этой моделью для расчета вероятности оттока для следующего клиента :

customer = {
    'customerid': '8879-zkjof',
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'no',
    'dependents': 'no',
    'tenure': 41,
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'dsl',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'yes',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'one_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'bank_transfer_(automatic)',
    'monthlycharges': 79.85,
    'totalcharges': 3320.75
}

In [8]:
# Делаем прогноз:

df = pd.DataFrame([customer])
y_pred = predict(df, dv, model)
y_pred[0].item()

0.05960531889903496

Вероятность оттока для этого клиента составляет 0.0596

In [9]:
# Функция для применения прогнозов в проверочном наборе. Становится неэффективной для применения к одному клиенту
def predict(df, dv, model):
    cat = df[categorical + numerical].to_dict(orient='rows')
    X = dv.transform(cat)
    y_pred = model.predict_proba(X)[:, 1]
    return y_pred.item()

In [10]:
# Чтобы избежать ненужного преобразования, переделаем функцию для прогноза по единственному клиенту
def predict_single(customer, dv, model):
    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[:, 1]
    return y_pred[0].item()

In [11]:
# Делаем прогноз:
predict_single(customer, dv, model)

0.05960531889903496

## Использование Pickle для сохранения и загрузки модели

### Сохранение модели

In [12]:
import pickle 

with open('churn-model.bin', 'wb') as f_out: # Указывает файл, в который мы хотим сохранить. 'wb' - запись в двоичном формате
    pickle.dump((dv, model), f_out) # Объект, который мы сохранили представляет собой кортеж из двух элементов: DictVectorizer и Модель

### Загрузка модели

In [13]:
# with open('churn-model.bin' , 'rb') as f_in: # открываем файл в режиме чтения
#     dv, model = pickle.load(f_in) # Загружает кортеж и распаковывает его

## POST-запрос к сервису Flask ('churn_serving_simple.py')

In [14]:
import requests
url = 'http://localhost:9696/predict'
response = requests.post(url, json=customer)
result = response.json()
result

{'churn': False, 'churn_probability': 0.05960531889903499}